In [4]:
import pandas as pd

In [5]:
train_fullname = 'data/train_data_public.csv'

In [6]:
class SimpleVocab:
    def __init__(self):
        labels = pd.read_csv(train_fullname)['BIO_anno'].to_list()
        labels = [label_line.split() for label_line in labels]
        all_tokens = [token for label_line in labels for token in label_line] # 这个写法每次看一遍都觉得震撼
        self.token_dict = {}
        self.token_array = []

        # O means none
        self.token_dict['O'] = len(self.token_array)
        self.token_array.append('O')

        for token in all_tokens:
            if token not in self.token_dict:
                self.token_dict[token] = len(self.token_array)
                self.token_array.append(token)

    def __call__(self, tokens):
        assert isinstance(tokens, (list, tuple, str))
        if isinstance(tokens, (list, tuple)):
            return [self(token) for token in tokens]
        else:
            return self.token_dict[tokens]

    def __len__(self):
        return len(self.token_array)

    def to_tokens(self, ids):
        assert isinstance(ids, (list, tuple, int))
        if isinstance(ids, (list, tuple)):
            return [self.to_tokens(idx) for idx in ids]
        else:
            return self.token_array[ids]

    def get_none_token(self):
        return 'O'

    def get_none_id(self):
        return self('O')

    def get_token_dict(self):
        return self.token_dict

# 将BIO编码序列转换为标注实体四元组

## 实体标记
将不同种类的BIO标签编码为int序列，对应如下

In [7]:
# BIO 标签被编码为int序列
lable_vocab = SimpleVocab()
print(lable_vocab.get_token_dict())

{'O': 0, 'B-BANK': 1, 'I-BANK': 2, 'B-COMMENTS_N': 3, 'I-COMMENTS_N': 4, 'B-COMMENTS_ADJ': 5, 'I-COMMENTS_ADJ': 6, 'B-PRODUCT': 7, 'I-PRODUCT': 8}


- `O`: $0$, 其他
- `B-BANK`: $1$, 银行实体开始
- `I-BANK`: $2$, 银行实体中间
- `B-COMMENTS_N`: $3$, 评论实体开始（名词）
- `I-COMMENTS_N`: $4$, 评论实体中间（名词）
- `B-COMMENTS_ADJ`: $5$, 评论实体开始（形容词）
- `I-COMMENTS_ADJ`: $6$, 评论实体中间（形容词）
- `B-PRODUCT`: $7$, 产品实体开始
- `I-PRODUCT`: $8$, 产品实体中间

需要根据这些标注，在一句话中找出识别出的实体，每个实体对应一个 `<实体类型，起始位置，结束位置>`
如上，实体类型有 `BANK`, `COMMENTS_N`, `COMMENTS_ADJ`, `PRODUCT`

![](1.png)


## 准确度判断

### 对每个样本
你会获得两个标记序列 `predict` 和 `target`
`predict` 是我们自己训练的模型做出的判断
`target` 是人工标注的正确答案

对两个序列，需要分别找到被如上方式标出的实体数量 |S| 和 |G|
以及标记正确（即类型，开头，结尾都匹配）的实体数量 |S∩G|
输出对每个样本的这三个数量